<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">OC SWITRS GIS Data Processing</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 1 - Feature Class Geoprocessing</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.1, January 2025</div>
</center></div>

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [ ]:
#%reset

Instantiating python libraries for the project

In [ ]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

Define and maitnain project, workspace, ArcGIS, and data-related variables

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project and Geodatabase Paths</h3>

Define the ArcGIS pro project variables

In [ ]:
# Environment variable for OneDrive path
onedriveFolder = os.getenv("OneDriveCommercial")

# OCSWITRS project path
projectFolder = os.path.join(onedriveFolder, "Documents", "OCSWITRS")

Which running environment this notebook is using? (1 = Visual Studio Code, 2 = ArcGIS Pro)

In [ ]:
runEnv = input("Enter the running environment (1=VSCode, 2=ArcGIS Pro): ")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">ArcGIS Pro Paths</h3>

ArcGIS pro related paths

In [ ]:
# OCSWITRS project AGP path
agpFolder = os.path.join(projectFolder, "AGPSWITRS")

# AGP APRX file name and path
aprxName = "AGPSWITRS.aprx"
aprxPath = os.path.join(agpFolder, aprxName)

# ArcGIS Pro project geodatabase and path
gdbName = "AGPSWITRS.gdb"
gdbPath = os.path.join(agpFolder, gdbName)

# ArcGIS pro project
if runEnv == "1":  # VSCode
    print("Running in VSCode (project = aprxPath)")
    aprx = arcpy.mp.ArcGISProject(aprxPath)
elif runEnv == "2":  # ArcGIS Pro
    print("Running in ArcGIS Pro (project = CURRENT)")
    aprx = arcpy.mp.ArcGISProject('CURRENT')

# Close all map views
aprx.closeViews()

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace
# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True
# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Folder Paths</h3>

In [ ]:
# Raw data folder path
rawDataFolder = os.path.join(projectFolder, "Data", "Raw")

# Maps folder path
mapsFolder = os.path.join(projectFolder, "Maps")

# Layers folder path
layersFolder = os.path.join(projectFolder, "Layers")
layersTemplate = os.path.join(layersFolder, "Templates")

# Layouts folder path
layoutsFolder = os.path.join(projectFolder, "Layouts")

# Notebooks folder path
notebooksFolder = os.path.join(projectFolder, "Notebooks")
codebookPath = os.path.join(projectFolder, "Data", "codebook", "cb.json")

Geodatabase feature datasets paths (directories)

In [ ]:
# RawData feature dataset in the geodatabase
gdbRawData = os.path.join(gdbPath, "RawData")

# RawData feature dataset in the geodatabase
gdbSupportingData = os.path.join(gdbPath, "SupportingData")

# AnalysisData feature dataset in the geodatabase
gdbAnalysisData = os.path.join(gdbPath, "AnalysisData")

# HotSpotData feature dataset in the geodatabase
gdbHotspotData = os.path.join(gdbPath, "HotSpotData")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Data Folder Paths</h3>

The most current raw data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [ ]:
# Add the start date of the raw data to a new python datetime object
dateStart = datetime(2012, 1, 1)

# Add the end date of the raw data to a new python datetime object
dateEnd = datetime(2024, 9, 30)

# Define time and date variables
timeZone = pytz.timezone("US/Pacific")
today = datetime.now(timeZone)
dateUpdated = today.strftime("%B %d, %Y")
timeUpdated = today.strftime("%I:%M %p")

Define date strings for metadata

In [ ]:
# String defining the years of the raw data
mdYears = f"{dateStart.year}-{dateEnd.year}"

# String defining the start and end dates of the raw data
mdDates = f"Data from {dateStart.strftime('%B %d, %Y')} to {dateEnd.strftime('%B %d, %Y')}"

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Codebook</h3>

In [ ]:
# Load the JSON file from directory and store it in a variable
with open(codebookPath) as jsonFile:
    codebook = json.load(jsonFile)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">JSON CIM Exports</h3>

Creating a function to export the CIM JSON files to disk.

In [ ]:
def export_cim(cimType, cimObject, cimName):
    """Export a CIM object to a file in both native (MAPX, PAGX, LYRX) and JSON CIM formats."""
    match cimType:
        # When the CIM object is a map
        case "map":
            # Export the CIM object to a MAPX file
            print(f"Exporting {cimName} map to MAPX...")
            cimObject.exportToMAPX(os.path.join(mapsFolder, cimName+".mapx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} map to JSON...\n")
            with open(os.path.join(mapsFolder, cimName+".mapx"), "r") as f:
                data = f.read()
            with open(os.path.join(mapsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layout
        case "layout":
            # Export the CIM object to a PAGX file
            print(f"Exporting {cimName} layout to PAGX...")
            cimObject.exportToPAGX(os.path.join(layoutsFolder, cimName+".pagx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layout to JSON...\n")
            with open(os.path.join(layoutsFolder, cimName+".pagx"), "r") as f:
                data = f.read()
            with open(os.path.join(layoutsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layer
        case "layer":
            # Export the CIM object to a LYRX file
            print(f"Exporting {cimName} layer to LYRX...")
            # Reformat the name of the output file
            for m in aprx.listMaps():
                for l in m.listLayers():
                    if l == cimObject:
                        cimNewName = m.name.title()+"Map-"+l.name.replace("OCSWITRS ", "")
            # Save the layer to a LYRX file
            arcpy.management.SaveToLayerFile(cimObject, os.path.join(layersFolder, cimNewName + ".lyrx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layer to JSON...\n")
            with open(os.path.join(layersFolder, cimNewName+".lyrx"), "r") as f:
                data = f.read()
            with open(os.path.join(layersFolder, cimNewName+".json"), "w") as f:
                f.write(data)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.3. ArcGIS Pro Workspace</h2>

Set the workspace and environment settings for the ArcGIS Pro project

In [ ]:
# Set the workspace and environment to the root of the project geodatabase
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True

# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.4. Map and Layout Lists</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Maps</h3>

In [ ]:
# List of maps to be created for the project
mapList = ["collisions", "crashes", "parties", "victims", "injuries", "fatalities", "fhs100m1km", "fhs150m2km", "fhs100m5km", "fhsRoads500ft", "ohsRoads500ft", "roadCrashes",
           "roadHotspots", "roadBuffers", "roadSegments", "roads", "pointFhs", "pointOhs", "popDens", "houDens", "areaCities", "areaBlocks", "summaries", "analysis", "regression"]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Layouts</h3>

In [ ]:
# List or layouts to be created for the project
layoutList = ["maps", "injuries", "hotspots", "roads", "points", "densities", "areas"]

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.5. Clean Up Data</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Feature Classes</h3>

Clean up the feature classes in the geodatabase for the Analysis and Hotspot Feature Datasets

In [ ]:
for d in ["AnalysisData", "HotSpotData"]:
    print(f"Dataset: {d}")
    for f in arcpy.ListFeatureClasses(feature_dataset = d):
        print(f"- Removing {f} feature class from the project...")
        arcpy.management.Delete(f)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Maps</h3>

Clean up the maps in the project structure

In [ ]:
for m in aprx.listMaps():
    print(f"- Removing {m.name} map from the project...")
    aprx.deleteItem(m)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Layouts</h3>

Clean up the layouts in the project structure

In [ ]:
for l in aprx.listLayouts():
    print(f"- Removing {l.name} layout from the project...")
    aprx.deleteItem(l)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [ ]:
# Save the project
aprx.save()

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">2. Geodatabase Operations</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.1. Raw Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

Paths to raw data geodatabase feature classes

In [ ]:
# Paths to raw data feature classes
victims = os.path.join(gdbRawData, "victims")
parties = os.path.join(gdbRawData, "parties")
crashes = os.path.join(gdbRawData, "crashes")
collisions = os.path.join(gdbRawData, "collisions")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Fields</h3>

Obtain a list of fields for each raw data geodatabase feature class

In [ ]:
# Fields for the raw data feature classes
victimsFields = [f.name for f in arcpy.ListFields(victims)] # victims field list
partiesFields = [f.name for f in arcpy.ListFields(parties)] # parties field list
crashesFields = [f.name for f in arcpy.ListFields(crashes)] # crashes field list
collisionsFields = [f.name for f in arcpy.ListFields(collisions)] # collisions field list

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Row Counts</h3>

Count rows in each of the raw data geodatabase feature classes

In [ ]:
# Get the count for the raw data feature classes
victimsCount = int(arcpy.management.GetCount(victims)[0])
partiesCount = int(arcpy.management.GetCount(parties)[0])
crashesCount = int(arcpy.management.GetCount(crashes)[0])
collisionsCount = int(arcpy.management.GetCount(collisions)[0])

print(f"\nRaw Data Counts:\n- Victims: {victimsCount:,}\n- Parties: {partiesCount:,}\n- Crashes: {crashesCount:,}\n- Collisions: {collisionsCount:,}")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Collisions Feature Class Aliases</h3>

Adding feature class alias for the collisions feature class

In [ ]:
# Collisions feature class alias
collisionsAlias = "OCSWITRS Collisions"

# Collisions feature class
arcpy.AlterAliasName(collisions, collisionsAlias)
print(f"Collisions: {arcpy.GetMessages()}")

Adding field aliases to the collisions feature class

In [ ]:
# Collisions field aliases
for f in collisionsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = collisions,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes Feature Class Aliases</h3>

Adding feature class alias for the crashes feature class

In [ ]:
# Crashes feature class alias
crashesAlias = "OCSWITRS Crashes"

# Crashes feature class
arcpy.AlterAliasName(crashes, crashesAlias)
print(f"Crashes: {arcpy.GetMessages()}")

Adding field aliases to the crashes feature class

In [ ]:
# Crashes field aliases
for f in crashesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = crashes,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Parties Feature Class Aliases</h3>

Adding feature class alias for the parties feature class

In [ ]:
# Parties feature class alias
partiesAlias = "OCSWITRS Parties"

# Parties feature class
arcpy.AlterAliasName(parties, partiesAlias)
print(f"Parties: {arcpy.GetMessages()}")

Adding field aliases to the parties feature class

In [ ]:
# Parties field aliases
for f in partiesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = parties,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Victims Feature Class Aliases</h3>

Adding feature class alias for the victims feature class

In [ ]:
# Victims feature class alias
victimsAlias = "OCSWITRS Victims"

# Victims feature class
arcpy.AlterAliasName(victims, victimsAlias)
print(f"Victims: {arcpy.GetMessages()}")

Adding field aliases to the victims feature class

In [ ]:
# Victims field aliases
for f in victimsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = victims,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.2. Supporting Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

Paths to the supporting data geodatabase feature classes

In [ ]:
# Paths to supporting data feature classes
boundaries = os.path.join(gdbSupportingData, "boundaries")
cities = os.path.join(gdbSupportingData, "cities")
censusBlocks = os.path.join(gdbSupportingData, "censusBlocks")
roads = os.path.join(gdbSupportingData, "roads")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Fields</h3>

Obtain the list fields of the supporting data geodatabase feature classes

In [ ]:
# Fields for the supporting data feature classes
boundariesFields = [f.name for f in arcpy.ListFields(boundaries)] # boundaries field list
citiesFields = [f.name for f in arcpy.ListFields(cities)] # cities field list
censusBlocksFields = [f.name for f in arcpy.ListFields(censusBlocks)] # censusBlocks field list
roadsFields = [f.name for f in arcpy.ListFields(roads)] # roads field list

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Row Counts</h3>

Count rows in each of the supporting data geodatabase feature classes

In [ ]:
# Get the count for the supporting data feature classes
boundariesCount = int(arcpy.management.GetCount(boundaries)[0])
citiesCount = int(arcpy.management.GetCount(cities)[0])
censusBlocksCount = int(arcpy.management.GetCount(censusBlocks)[0])
roadsCount = int(arcpy.management.GetCount(roads)[0])

# Print the counts
print(f"Supporting Data Counts:\n- Boundaries: {boundariesCount:,}\n- Cities: {citiesCount:,}\n- Census Blocks: {censusBlocksCount:,}\n- Roads: {roadsCount:,}")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Roads Feature Class</h3>

Adding feature class alias for the roads feature class

In [ ]:
# Roads feature class alias
roadsAlias = "OCSWITRS Roads"

# Roads feature class
arcpy.AlterAliasName(roads, roadsAlias)
print(f"Roads: {arcpy.GetMessages()}")

Adding field aliases to the roads feature class

In [ ]:
# Roads field aliases
for f in roadsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = roads,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Census Blocks Feature Class</h3>

Adding feature class alias for the census blocks feature class

In [ ]:
# Census Blocks feature class alias
censusBlocksAlias = "OCSWITRS Census Blocks"

# Census Blocks feature class
arcpy.AlterAliasName(censusBlocks, censusBlocksAlias)
print(f"USC 2020 Census Blocks: {arcpy.GetMessages()}")

Adding field aliases to the census blocks feature class

In [ ]:
# Census Blocks field aliases
for f in censusBlocksFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = censusBlocks,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Cities Feature Class</h3>

Adding feature class alias for the cities feature class

In [ ]:
# Cities feature class alias
citiesAlias = "OCSWITRS Cities"

# Cities feature class
arcpy.AlterAliasName(cities, citiesAlias)
print(f"Cities: {arcpy.GetMessages()}")

Adding field aliases to the cities feature class

In [ ]:
# Cities field aliases
for f in citiesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = cities,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Boundaries Feature Class</h3>

Adding feature class alias for the boundaries feature class

In [ ]:
# Boundaries feature class alias
boundariesAlias = "OCSWITRS Boundaries"

# Boundaries feature class
arcpy.AlterAliasName(boundaries, boundariesAlias)
print(f"Boundaries: {arcpy.GetMessages()}")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [ ]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Data Enrichment Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

In [ ]:
collisions1 = os.path.join(gdbRawData, "collisions1")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Joins</h3>

In [ ]:
# Join the collisions feature class with the censusBlocks feature class
arcpy.analysis.SpatialJoin(
    target_features = collisions,
    join_features = censusBlocks,
    out_feature_class = collisions1,
    join_operation = "JOIN_ONE_TO_ONE",
    join_type = "KEEP_ALL",
    match_option = "INTERSECT",
    search_radius = None,
    distance_field_name = None,
    match_fields = None
)



In [ ]:
arcpy.analysis.SpatialJoin(
    target_features="OCSWITRS Crashes",
    join_features="OCSWITRS Census Blocks",
    out_feature_class=r"C:\Users\ktalexan\AppData\Local\Temp\ArcGISProTemp25272\Untitled\Default.gdb\OCSWITRSCrashes_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='caseId "Case ID" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,caseId,-1,-1;cid "Crash ID" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,cid,0,254;crashTag "Crash Tag" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,crashTag,-1,-1;crashesCaseTag "Crashes Tag" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,crashesCaseTag,-1,-1;crashesCidCount "Total Collisions in Crashes Dataset" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,crashesCidCount,-1,-1;city "City" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,city,0,254;dateDatetime "Crash Date and Time" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateDatetime,-1,-1;dateYear "Crash Year" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateYear,-1,-1;dateQuarter "Crash Quarter" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateQuarter,-1,-1;dateMonth "Crash Month" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateMonth,-1,-1;dateWeek "Crash Week" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateWeek,-1,-1;dateDay "Crash Day" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateDay,-1,-1;dateProcess "Process Date" true true false 8 Date 0 0,First,#,OCSWITRS Crashes,dateProcess,-1,-1;dtYear "Crash Year" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtYear,-1,-1;dtQuarter "Crash Quarter" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,dtQuarter,0,254;dtMonth "Crash Month" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,dtMonth,0,254;dtYearWeek "Crash Year Week" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtYearWeek,-1,-1;dtWeekDay "Crash Weekday" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,dtWeekDay,0,254;dtMonthDay "Crash Month Day" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtMonthDay,-1,-1;dtYearDay "Crash Year Day" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtYearDay,-1,-1;dtHour "Crash Hour" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtHour,-1,-1;dtMinute "Crash Minute" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,dtMinute,-1,-1;dtDst "Crash Daylight Saving Time" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,dtDst,0,254;dtZone "Crash Time Zone" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,dtZone,0,254;collDate "Collision Date Reported" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,collDate,0,254;collTime "Collision Time Reported" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,collTime,-1,-1;accidentYear "Crash Year" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,accidentYear,-1,-1;processDate "Processing Date" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,processDate,0,254;collTimeIntervals "Crash Time Intervals" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,collTimeIntervals,0,254;rushHours "Rush Hours" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,rushHours,0,254;rushHoursBin "Rush Hours Binary" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,rushHoursBin,0,254;collSeverity "Collision Severity" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,collSeverity,0,254;collSeverityNum "Collision Severity Numeric" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,collSeverityNum,-1,-1;collSeverityBin "Collision Severity Binary" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,collSeverityBin,0,254;collSeverityRank "Collision Severity Rank" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,collSeverityRank,0,254;collSeverityRankNum "Collision Severity Rank Numeric" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,collSeverityRankNum,-1,-1;indSevere "Severe Injury Indicator" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,indSevere,0,254;indFatal "Fatal Injury Indicator" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,indFatal,0,254;indMulti "Multiple Injury Indicator" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,indMulti,0,254;partyCount "Party Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,partyCount,-1,-1;victimCount "Victim Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,victimCount,-1,-1;numberKilled "Killed Victims" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,numberKilled,-1,-1;numberInj "Injured Victims" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,numberInj,-1,-1;countSevereInj "Severe Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countSevereInj,-1,-1;countVisibleInj "Visible Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countVisibleInj,-1,-1;countComplaintPain "Complaint of Pain Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countComplaintPain,-1,-1;countCarKilled "Number of Killed Car Victims" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countCarKilled,-1,-1;countCarInj "Number of Injured Car Victims" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countCarInj,-1,-1;countPedKilled "Pedestrian Killed Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countPedKilled,-1,-1;countPedInj "Pedestrian Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countPedInj,-1,-1;countBicKilled "Bicyclist Killed Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countBicKilled,-1,-1;countBicInj "Bicyclist Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countBicInj,-1,-1;countMcKilled "Motorcyclist Killed Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countMcKilled,-1,-1;countMcInj "Motorcyclist Injury Count" true true false 4 Long 0 0,First,#,OCSWITRS Crashes,countMcInj,-1,-1;primaryCollFactor "Primary Collision Factor" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,primaryCollFactor,0,254;typeOfColl "Type of Crash" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,typeOfColl,0,254;pedAccident "Pedestrian Crash" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pedAccident,0,254;bicAccident "Bicycle Crash" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,bicAccident,0,254;mcAccident "Motorcycle Crash" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,mcAccident,0,254;truckAccident "Truck Crash" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,truckAccident,0,254;hitAndRun "Hit and Run" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,hitAndRun,0,254;hitAndRunBin "Hit and Run Binary" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,hitAndRunBin,0,254;alcoholInvolved "Alcohol Involved" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,alcoholInvolved,0,254;juris "Jurisdiction" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,juris,0,254;officerId "Officer ID" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,officerId,0,254;reportingDistrict "Reporting District" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,reportingDistrict,0,254;chpShift "CHP Shift" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,chpShift,0,254;cntyCityLoc "County City Location" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,cntyCityLoc,0,254;specialCond "Special Condition" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,specialCond,0,254;beatType "Beat Type" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,beatType,0,254;chpBeatType "CHP Beat Type" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,chpBeatType,0,254;chpBeatClass "CHP Beat Class" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,chpBeatClass,0,254;beatNumber "Beat Number" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,beatNumber,0,254;primaryRd "Primary Road" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,primaryRd,0,254;secondaryRd "Secondary Road" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,secondaryRd,0,254;distance "Distance" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,distance,-1,-1;direction "Direction" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,direction,0,254;intersection "Intersection" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,intersection,0,254;weather1 "Weather 1" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,weather1,0,254;weather2 "Weather 2" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,weather2,0,254;weatherComb "Combined Weather" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,weatherComb,0,254;roadSurface "Road Surface" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,roadSurface,0,254;roadCond1 "Road Condition 1" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,roadCond1,0,254;roadCond2 "Road Condition 2" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,roadCond2,0,254;lighting "Lighting" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,lighting,0,254;controlDevice "Control Device" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,controlDevice,0,254;stateHwyInd "State Highway Indicator" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,stateHwyInd,0,254;sideOfHwy "Side of Highway" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,sideOfHwy,0,254;towAway "Tow Away" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,towAway,0,254;pcfCodeOfViol "PCF Violation Code" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pcfCodeOfViol,0,254;pcfViolCategory "PCF Violation Category" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pcfViolCategory,0,254;pcfViolation "PCF Violation" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pcfViolation,0,254;pcfViolSubsection "PCF Violation Subsection" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pcfViolSubsection,0,254;mviw "Motor Vehicle Involved With" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,mviw,0,254;pedAction "Pedestrian Action" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,pedAction,0,254;notPrivateProperty "Not Private Property" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,notPrivateProperty,0,254;stwdVehTypeAtFault "Statewide Vehicle Type at Fault" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,stwdVehTypeAtFault,0,254;chpVehTypeAtFault "CHP Vehicle Type at Fault" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,chpVehTypeAtFault,0,254;primaryRamp "Primary Ramp" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,primaryRamp,0,254;secondaryRamp "Secondary Ramp" true true false 255 Text 0 0,First,#,OCSWITRS Crashes,secondaryRamp,0,254;latitude "GPS Latitude" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,latitude,-1,-1;longitude "GPS Longitude" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,longitude,-1,-1;pointX "X Coordinate Location" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,pointX,-1,-1;pointY "Y Coordinate Location" true true false 8 Double 0 0,First,#,OCSWITRS Crashes,pointY,-1,-1;severityBin "Severity Binary" true true false 2 Short 0 0,First,#,OCSWITRS Crashes,severityBin,-1,-1;GEOID "Geographic Identifier" true true false 16 Text 0 0,First,#,OCSWITRS Census Blocks,GEOID,0,15;SUFFIX "Suffix" true true false 1 Text 0 0,First,#,OCSWITRS Census Blocks,SUFFIX,0,0;NAME "Name" true true false 11 Text 0 0,First,#,OCSWITRS Census Blocks,NAME,0,10;populationTotal "Total Population" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,populationTotal,-1,-1;housingUnitsTotal "Total Housing Units" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,housingUnitsTotal,-1,-1;householdsTotal "Total Households" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,householdsTotal,-1,-1;housingUnitsVacant "Vacant Housing Units" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,housingUnitsVacant,-1,-1;housingUnitsMortgage "Housing Units with Mortgage" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,housingUnitsMortgage,-1,-1;howsingUnitsOwned "Owned housing units free and clear" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,howsingUnitsOwned,-1,-1;housingUnitsRenterOccupied "Renter Occupied Housing Units" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,housingUnitsRenterOccupied,-1,-1;vacancyRate "Vacancy Rate" true true false 8 Double 0 0,First,#,OCSWITRS Census Blocks,vacancyRate,-1,-1;housingUnitsOwnerOccupied "Owner Occupied Housing Units" true true false 4 Long 0 0,First,#,OCSWITRS Census Blocks,housingUnitsOwnerOccupied,-1,-1;homeownershipRate "Homeownership Rate" true true false 8 Double 0 0,First,#,OCSWITRS Census Blocks,homeownershipRate,-1,-1;populationDensity "Population Density" true true false 8 Double 0 0,First,#,OCSWITRS Census Blocks,populationDensity,-1,-1;housingDensity "Housing Density" true true false 8 Double 0 0,First,#,OCSWITRS Census Blocks,housingDensity,-1,-1',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Analysis Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete all Old Analysis Feature Classes</h3>

Loop through all analysis data feature dataset and delete all feature classes

In [ ]:
for f in arcpy.ListFeatureClasses(feature_dataset = "AnalysisData"):
    print(f"Deleting {f}...")
    arcpy.Delete_management(f)
    print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Major Roads</h3>

Separate the primary and secondary roads from the local roads

In [ ]:
# Output feature class for the major roads
roadsMajor = os.path.join(gdbAnalysisData, "roadsMajor")

# Select the major (primary and secondary) roads from the roads feature class
arcpy.analysis.Select(
    in_features = roads,
    out_feature_class = roadsMajor,
    where_clause = "roadCat = 'Primary' Or roadCat = 'Secondary'"
)
print(arcpy.GetMessages())

Add feature class alias for the major roads feature class

In [ ]:
# Define the major roads layer alias and modify the feature class alias
roadsMajorAlias = "OCSWITRS Major Roads"
arcpy.AlterAliasName(roadsMajor, roadsMajorAlias)
print(arcpy.GetMessages())

Obtain the list of fields for the major roads feature class

In [ ]:
roadsMajorFields = [f.name for f in arcpy.ListFields(roadsMajor)] # roadsMajor field list

# Field Aliases for the major roads feature class
for f in arcpy.ListFields(roadsMajor):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Major Road Buffers</h3>

Create road buffers for the primary and secondary roads

In [ ]:
# Output feature class for the major roads buffers
roadsMajorBuffers = os.path.join(gdbAnalysisData, "roadsMajorBuffers")

# Buffer the major roads feature class by 250 meters (on each side)
arcpy.analysis.Buffer(
    in_features = roadsMajor,
    out_feature_class = roadsMajorBuffers,
    buffer_distance_or_field = "250 Meters",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "NONE",
    dissolve_field = None,
    method = "PLANAR"
)
print(arcpy.GetMessages())

Add feature class alias for the major road buffers feature class

In [ ]:
# Define the major roads buffers layer alias and modify the feature class alias
roadsMajorBuffersAlias = "OCSWITRS Major Roads Buffers"
arcpy.AlterAliasName(roadsMajorBuffers, roadsMajorBuffersAlias)
print(arcpy.GetMessages())

Obtain the list of fields for the major road buffers feature class

In [ ]:
roadsMajorBuffersFields = [f.name for f in arcpy.ListFields(roadsMajorBuffers)] # roadsMajorBuffers field list

# Field Aliases for the major roads buffers feature class
for f in arcpy.ListFields(roadsMajorBuffers):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Major Road Buffers</h3>

Create a summary for each of the road buffers that contains statistics and counts of crash collision data

In [ ]:
# Output feature class for the summarized major roads buffers
roadsMajorBuffersSum = os.path.join(gdbAnalysisData, "roadsMajorBuffersSum")

# Summarize the major roads buffers feature class by key crashes attributes
arcpy.analysis.SummarizeWithin(
    in_polygons = roadsMajorBuffers,
    in_sum_features = crashes,
    out_feature_class = roadsMajorBuffersSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields = [
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)
print(arcpy.GetMessages())

Add feature class alias for the summarized major road buffers feature class

In [ ]:
# Define the major roads buffers summary layer alias and modify the feature class alias
roadsMajorBuffersSumAlias = "OCSWITRS Major Roads Buffers Summary"
arcpy.AlterAliasName(roadsMajorBuffersSum, roadsMajorBuffersSumAlias)
print(arcpy.GetMessages())

Obtain the fields for the summarized major road buffers feature class

In [ ]:
roadsMajorBuffersSumFields = [f.name for f in arcpy.ListFields(roadsMajorBuffersSum)] # roadsMajorBuffersSum field list

# Field Aliases for the major roads buffers summary feature class
for f in arcpy.ListFields(roadsMajorBuffersSum):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Points 1,000 ft along major road lines</h3>

Generate points every 1,000 feet along the major road lines

In [ ]:
# Create a path for the new summarized major road buffers feature class
roadsMajorPointsAlongLines = os.path.join(gdbAnalysisData, "roadsMajorPointsAlongLines")

arcpy.management.GeneratePointsAlongLines(
    Input_Features = roadsMajor,
    Output_Feature_Class = roadsMajorPointsAlongLines,
    Point_Placement = "DISTANCE",
    Distance = "1000 Feet",
    Percentage = None,
    Include_End_Points = "NO_END_POINTS",
    Add_Chainage_Fields = "NO_CHAINAGE",
    Distance_Field=None,
    Distance_Method = "PLANAR"
)

Add feature class alias for the points along major road lines feature class

In [ ]:
# Define the major roads points along lines layer alias and modify the feature class alias
roadsMajorPointsAlongLinesAlias = "OCSWITRS Major Roads Points Along Lines"
arcpy.AlterAliasName(roadsMajorPointsAlongLines, roadsMajorPointsAlongLinesAlias)
print(arcpy.GetMessages())

Obtain the fields for the points along major road lines feature class

In [ ]:
roadsMajorPointsAlongLinesFields = [f.name for f in arcpy.ListFields(roadsMajorPointsAlongLines)] # roadsMajorPointsAlongLines field list

# Field Aliases for the major roads points along lines feature class
for f in arcpy.ListFields(roadsMajorPointsAlongLines):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Split Road Segments 1,000ft apart</h3>

Split road segments at the points (1,000 feet apart)

In [ ]:
# Create a path for the new split major roads feature class
roadsMajorSplit = os.path.join(gdbAnalysisData, "roadsMajorSplit")

# Split the major roads at the points along the lines
arcpy.management.SplitLineAtPoint(
    in_features = roadsMajor,
    point_features = roadsMajorPointsAlongLines,
    out_feature_class = roadsMajorSplit,
    search_radius = "1000 Feet"
)

Add feature class alias for the split road segments feature class

In [ ]:
# Define the major roads split layer alias and modify the feature class alias
roadsMajorSplitAlias = "OCSWITRS Major Roads Split"
arcpy.AlterAliasName(roadsMajorSplit, roadsMajorSplitAlias)
print(arcpy.GetMessages())

Obtain the fields for the split road segments feature class

In [ ]:
roadsMajorSplitFields = [f.name for f in arcpy.ListFields(roadsMajorSplit)] # roadsMajorSplit field list

# Field Aliases for the major roads split feature class
for f in arcpy.ListFields(roadsMajorSplit):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Buffers 500ft around road segments</h3>

Create buffers (500 ft) around the road segments (1,000 feet)

In [ ]:
# Create a path for the new split major roads feature class
roadsMajorSplitBuffer = os.path.join(gdbAnalysisData, "roadsMajorSplitBuffer")

# Buffer the split major roads by 500 feet
arcpy.analysis.Buffer(
    in_features = roadsMajorSplit,
    out_feature_class = roadsMajorSplitBuffer,
    buffer_distance_or_field = "500 Feet",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "NONE",
    dissolve_field = None,
    method = "PLANAR"
)

Add feature class alias for the road segment buffers feature class

In [ ]:
# Define the major roads split buffer layer alias and modify the feature class alias
roadsMajorSplitBufferAlias = "OCSWITRS Major Roads Split Buffer"
arcpy.AlterAliasName(roadsMajorSplitBuffer, roadsMajorSplitBufferAlias)
print(arcpy.GetMessages())

Obtain the fields for the road segment buffers feature class

In [ ]:
roadsMajorSplitBufferFields = [f.name for f in arcpy.ListFields(roadsMajorSplitBuffer)] # roadsMajorSplitBuffer field list

# Field Aliases for the major roads split buffer feature class
for f in arcpy.ListFields(roadsMajorSplitBuffer):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize road segments buffers</h3>

Summarize the crash collision data for each of the road segments

In [ ]:
# Create a path for the new summarized major road buffers feature class
roadsMajorSplitBufferSum = os.path.join(gdbAnalysisData, "roadsMajorSplitBufferSum")

# Summarize the data within the major road buffers from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = roadsMajorSplitBuffer,
    in_sum_features = crashes,
    out_feature_class = roadsMajorSplitBufferSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

Add feature class alias for the summarized road segments buffers feature class

In [ ]:
# Define the major roads split buffer summary layer alias and modify the feature class alias
roadsMajorSplitBufferSumAlias = "OCSWITRS Major Roads Split Buffer Summary"
arcpy.AlterAliasName(roadsMajorSplitBufferSum, roadsMajorSplitBufferSumAlias)
print(arcpy.GetMessages())

Obtain the fields for the summarized road segments buffers feature class

In [ ]:
roadsMajorSplitBufferSumFields = [f.name for f in arcpy.ListFields(roadsMajorSplitBufferSum)] # roadsMajorSplitBufferSum field list

# Field Aliases for the major roads split buffer summary feature class
for f in arcpy.ListFields(roadsMajorSplitBufferSum):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Census Blocks</h3>

Create a summary for each of the Census blocks that contains statistics and counts of crash collision data

In [ ]:
# Create a path for the new summarized US 2020 Census Blocks feature class
censusBlocksSum = os.path.join(gdbAnalysisData, "censusBlocksSum")

arcpy.analysis.SummarizeWithin(
    in_polygons = censusBlocks,
    in_sum_features = crashes,
    out_feature_class = censusBlocksSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields = [
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

Add feature class alias for the summarized Census blocks feature class

In [ ]:
# Define the US 2020 Census Blocks summary layer alias and modify the feature class alias
censusBlocksSumAlias = "OCSWITRS Census Blocks Summary"
arcpy.AlterAliasName(censusBlocksSum, censusBlocksSumAlias)
print(arcpy.GetMessages())

Obtain the fields for the summarized Census blocks feature class

In [ ]:
cenBlocksSumFields = [f.name for f in arcpy.ListFields(censusBlocksSum)] # cenBlocksSum field list

# Field Aliases for the US 2020 Census Blocks summary feature class
for f in arcpy.ListFields(censusBlocksSum):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Cities</h3>

Create a summary for each of the cities that contains statistics and counts of crash collision data

In [ ]:
# Create a path for the new summarized US 2020 Census Blocks feature class
citiesSum = os.path.join(gdbAnalysisData, "citiesSum")

# Summarize the data within the cities from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = cities,
    in_sum_features = crashes,
    out_feature_class = citiesSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

Add feature class alias for the summarized cities feature class

In [ ]:
# Define the cities summary layer alias and modify the feature class alias
citiesSumAlias = "OCSWITRS Cities Summary"
arcpy.AlterAliasName(citiesSum, citiesSumAlias)
print(arcpy.GetMessages())

Obtain the fields for the summarized cities feature class

In [ ]:
citiesSumFields = [f.name for f in arcpy.ListFields(citiesSum)] # citiesSum field list

# Field Aliases for the cities summary feature class
for f in arcpy.ListFields(citiesSum):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes within 500ft from Major Roads</h3>

Select all crashes that are within 500 ft of the major roads

In [ ]:
# Create a path for the new feature class
crashes500ftFromMajorRoads = os.path.join(gdbAnalysisData, "crashes500ftFromMajorRoads")

# Select the crashes within 500 feet of the major roads and store it in a temporary layer
tempLyr = arcpy.management.SelectLayerByLocation(
    in_layer = crashes,
    select_features = roadsMajor,
    search_distance = "500 Feet",
    selection_type = "NEW_SELECTION",
    invert_spatial_relationship = "NOT_INVERT"
)

# Export the selected crashes to a new feature class
arcpy.conversion.ExportFeatures(
    in_features = tempLyr,
    out_features = crashes500ftFromMajorRoads,
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
)
print(arcpy.GetMessages())

# Delete the temporary layer
arcpy.management.Delete(tempLyr)

Add feature class alias for the crashes within 500 ft from major roads feature class

In [ ]:
# Define the crashes 500 feet from major roads layer alias and modify the feature class alias
crashes500ftFromMajorRoadsAlias = "OCSWITRS Crashes 500 Feet from Major Roads"
arcpy.AlterAliasName(crashes500ftFromMajorRoads, crashes500ftFromMajorRoadsAlias)
print(arcpy.GetMessages())

Obtain the fields for the crashes within 500 ft from major roads feature class

In [ ]:
crashes500ftFromMajorRoadsFields = [f.name for f in arcpy.ListFields(crashes500ftFromMajorRoads)] # crashes500ftFromMajorRoads field list

# Field Aliases for the crashes 500 feet from major roads feature class
for f in arcpy.ListFields(crashes500ftFromMajorRoads):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes (Collision Severity) Exploratory Regression</h3>

Generate a collision severity binary indicator to crashes dataset

In [ ]:
# Add collision severity binary indicator to crashes
arcpy.management.CalculateField(
    in_table = crashes,
    field = "severityBin",
    expression = "sevbin(!collSeverityBin!)",
    expression_type = "PYTHON3",
    code_block = """def sevbin(x):
    if x == "Severe or fatal":
        return 1
    elif x == "None, minor or pain":
        return 0""",
    field_type = "SHORT",
    enforce_domains = "NO_ENFORCE_DOMAINS"
)

Add a field alias for the collision severity binary indicator

In [ ]:
arcpy.management.AlterField(
    in_table = crashes,
    field = "severityBin",
    new_field_alias = "Severity Binary"
)

Perform exploratory regression to predict the binary severity bin

In [ ]:
arcpy.stats.ExploratoryRegression(
    Input_Features=crashes,
    Dependent_Variable="severityBin",
    Candidate_Explanatory_Variables="accidentYear;collSeverityNum;collSeverityRankNum;partyCount;victimCount;numberKilled;numberInj;countSevereInj;countVisibleInj;countComplaintPain;countCarKilled;countCarInj;countPedKilled;countPedInj;countBicKilled;countBicInj;countMcKilled;countMcInj",
    Weights_Matrix_File=None,
    Output_Report_File=None,
    Output_Results_Table=None,
    Maximum_Number_of_Explanatory_Variables=5,
    Minimum_Number_of_Explanatory_Variables=1,
    Minimum_Acceptable_Adj_R_Squared=0.5,
    Maximum_Coefficient_p_value_Cutoff=0.05,
    Maximum_VIF_Value_Cutoff=7.5,
    Minimum_Acceptable_Jarque_Bera_p_value=0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value=0.1
)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [ ]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.4. Hotspot Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete all Old Hotspot Feature Classes</h3>

Loop through all hotspot data feature dataset and delete all feature classes

In [ ]:
for f in arcpy.ListFeatureClasses(feature_dataset = "HotSpotData"):
    print(f"Deleting {f}...")
    arcpy.Delete_management(f)
    print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Hot Spots (Crashes, Collision Severity)</h3>

Create a hot spot analysis for the crash collision data (collision severity)

In [ ]:
# Create a path for the new crashes hot spots feature class
crashesHotspots = os.path.join(gdbHotspotData, "crashesHotspots")

# Create hot spots points
arcpy.stats.HotSpots(
    Input_Feature_Class = crashes,
    Input_Field = "collSeverityNum",
    Output_Feature_Class = crashesHotspots,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

Add feature class alias for the hot spots (crashes, collision severity) feature class

In [ ]:
# Define the crashes hot spots layer alias and modify the feature class alias
crashesHotspotsAlias = "OCSWITRS Crashes Hot Spots"
arcpy.AlterAliasName(crashesHotspots, crashesHotspotsAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (crashes, collision severity) feature class

In [ ]:
crashesHotspotsFields = [f.name for f in arcpy.ListFields(crashesHotspots)] # crashesHotspots field list

# Field Aliases for the crashes hot spots feature class
for f in arcpy.ListFields(crashesHotspots):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Optimized Hot Spots (Crashes, Collision Severity, 1,000 m)</h3>

Optimized hot spot analysis for the crash collision data (collision severity)

In [ ]:
# Create a path for the new optimized crashes hot spots feature class
crashesOptimizedHotspots = os.path.join(gdbHotspotData, "crashesOptimizedHotspots")

# Perform Optimized Hot Spot Analysis on the crashes data
arcpy.stats.OptimizedHotSpotAnalysis(
    Input_Features = crashes,
    Output_Features = crashesOptimizedHotspots,
    Analysis_Field = "collSeverityNum",
    Incident_Data_Aggregation_Method = "COUNT_INCIDENTS_WITHIN_FISHNET_POLYGONS",
    Bounding_Polygons_Defining_Where_Incidents_Are_Possible = None,
    Polygons_For_Aggregating_Incidents_Into_Counts = None,
    Density_Surface = None,
    Cell_Size = None,
    Distance_Band = "1000 Meters"
)

Add feature class alias for the optimized hot spots (crashes, collision severity, 1,000 m) feature class

In [ ]:
# Define the optimized crashes hot spots layer alias and modify the feature class alias
crashesOptimizedHotspotsAlias = "OCSWITRS Crashes Optimized Hot Spots"
arcpy.AlterAliasName(crashesOptimizedHotspots, crashesOptimizedHotspotsAlias)
print(arcpy.GetMessages())

Obtain the fields for the optimized hot spots (crashes, collision severity, 1,000 m) feature class

In [ ]:
crashesOptimizedHotspotsFields = [f.name for f in arcpy.ListFields(crashesOptimizedHotspots)] # crashesOptimizedHotspots field list

# Field Aliases for the optimized crashes hot spots feature class
for f in arcpy.ListFields(crashesOptimizedHotspots):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 100m bins, 1km neighbors)</h3>

Find hot spots for the crash collision data using 100 m bins and 1 km neighborhood radius (328 ft/ 0.621 mi)

In [ ]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots100m1km = os.path.join(gdbHotspotData, "crashesFindHotspots100m1km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots100m1km,
    bin_size = "100 Meters",
    neighborhood_size = "1 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

Add feature class alias for the hot spots (crashes, 100m bins, 1km neighbors) feature class

In [ ]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots100m1kmAlias = "OCSWITRS Crashes Find Hot Spots 100m 1km"
arcpy.AlterAliasName(crashesFindHotspots100m1km, crashesFindHotspots100m1kmAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (crashes, 100m bins, 1km neighbors) feature class

In [ ]:
crashesFindHotspots100m1kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots100m1km)] # crashesFindHotspots100m1km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots100m1km):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 150m bins, 2km neighbors)</h3>

Find hot spots for the crash collision data using 150 m bins and 2 km neighborhood radius (492 ft/ 1.24 mi)

In [ ]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots150m2km = os.path.join(gdbHotspotData, "crashesFindHotspots150m2km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots150m2km,
    bin_size = "150 Meters",
    neighborhood_size = "2 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

Add feature class alias for the hot spots (crashes, 150m bins, 2km neighbors) feature class

In [ ]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots150m2kmAlias = "OCSWITRS Crashes Find Hot Spots 150m 2km"
arcpy.AlterAliasName(crashesFindHotspots150m2km, crashesFindHotspots150m2kmAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (crashes, 150m bins, 2km neighbors) feature class

In [ ]:
crashesFindHotspots150m2kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots150m2km)] # crashesFindHotspots150m2km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots150m2km):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 100m bins, 5km neighbors)</h3>

Find hot spots for the crash collision data using 100 m bins and 5 km neighborhood radius (328 ft/ 3.11 mi)

In [ ]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots100m5km = os.path.join(gdbHotspotData, "crashesFindHotspots100m5km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots100m5km,
    bin_size = "100 Meters",
    neighborhood_size = "5 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

Add feature class alias for the hot spots (crashes, 100m bins, 5km neighbors) feature class

In [ ]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots100m5kmAlias = "OCSWITRS Crashes Find Hot Spots 100m 5km"
arcpy.AlterAliasName(crashesFindHotspots100m5km, crashesFindHotspots100m5kmAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (crashes, 100m bins, 5km neighbors) feature class

In [ ]:
crashesFindHotspots100m5kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots100m5km)] # crashesFindHotspots100m5km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots100m5km):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Hot Spots (Proximity to Major Roads, 500ft)</h3>

Hot spot points within 500 feet of major roads

In [ ]:
# Create a path for the new hot spots within 500 mt from major roads feature class
crashesHotspots500ftFromMajorRoads = os.path.join(gdbHotspotData, "crashesHotspots500ftFromMajorRoads")

arcpy.stats.HotSpots(
    Input_Feature_Class = crashes500ftFromMajorRoads,
    Input_Field="collSeverityNum",
    Output_Feature_Class = crashesHotspots500ftFromMajorRoads,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

Add feature class alias for the hot spots (proximity to major roads, 500ft) feature class

In [ ]:
# Define the crashes hot spots 500 feet from major roads layer alias and modify the feature class alias
crashesHotspots500ftFromMajorRoadsAlias = "OCSWITRS Crashes Hot Spots 500 Feet from Major Roads"
arcpy.AlterAliasName(crashesHotspots500ftFromMajorRoads, crashesHotspots500ftFromMajorRoadsAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (proximity to major roads, 500ft) feature class

In [ ]:
crashesHotspots500ftFromMajorRoadsFields = [f.name for f in arcpy.ListFields(crashesHotspots500ftFromMajorRoads)] # crashesHotspots500ftFromMajorRoads field list

# Field Aliases for the crashes hot spots 500 feet from major roads feature class
for f in arcpy.ListFields(crashesHotspots500ftFromMajorRoads):
    print(f"{f.name} ({f.aliasName})")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Proximity to Major Roads, 500ft)</h3>

Find hot spots within 500 feet from major roads

In [ ]:
# Create a path for the new hot spots within 500 ft from major roads feature class
crashesFindHotspots500ftMajorRoads500ft1mi = os.path.join(gdbHotspotData, "crashesFindHotspots500ftMajorRoads500ft1mi")

arcpy.gapro.FindHotSpots(
    point_layer = crashes500ftFromMajorRoads,
    out_feature_class = crashesFindHotspots500ftMajorRoads500ft1mi,
    bin_size="500 Feet",
    neighborhood_size = "1 Miles",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference=None
)

Add feature class alias for the hot spots (proximity to major roads, 500ft) feature class

In [ ]:
# Define the crashes find hot spots 500 feet from major roads layer alias and modify the feature class alias
crashesFindHotspots500ftMajorRoads500ft1miAlias = "OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi"
arcpy.AlterAliasName(crashesFindHotspots500ftMajorRoads500ft1mi, crashesFindHotspots500ftMajorRoads500ft1miAlias)
print(arcpy.GetMessages())

Obtain the fields for the hot spots (proximity to major roads, 500ft) feature class

In [ ]:
crashesFindHotspots500ftMajorRoads500ft1miFields = [f.name for f in arcpy.ListFields(crashesFindHotspots500ftMajorRoads500ft1mi)] # crashesFindHotspots500ftMajorRoads500ft1mi field list

# Field Aliases for the crashes find hot spots 500 feet from major roads feature class
for f in arcpy.ListFields(crashesFindHotspots500ftMajorRoads500ft1mi):
    print(f"{f.name} ({f.aliasName})")

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">3. Feature Class Metadata Processing</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.1. Collisions Metadata</h2>

Create a new metadata object for the collisions feature class

In [ ]:
# Define key metadata attributes for the Collisions feature class
mdoCollisions = md.Metadata()
mdoCollisions.title = "OCSWITRS Combined Collisions Points"
mdoCollisions.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCollisions.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoCollisions.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoCollisions.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCollisions.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCollisions.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the collisions metadata object to the collisions feature class

In [ ]:
# Apply the metadata object to the collisions feature class
mdCollisions = md.Metadata(collisions)
if not mdCollisions.isReadOnly:
    mdCollisions.copy(mdoCollisions)
    mdCollisions.save()
    print(f"Metadata updated for {collisionsAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Crashes Metadata</h2>

Create a new metadata object for the crashes feature class

In [ ]:
# Define key metadata attributes for the Crashes feature class
mdoCrashes = md.Metadata()
mdoCrashes.title = "OCSWITRS Crashes Points"
mdoCrashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCrashes.summary = f"Statewide Integrated Traffic Records System (SWITRS) Crash Data for Orange County, California ({mdYears})"
mdoCrashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on crashes</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoCrashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCrashes.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCrashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the crashes metadata object to the crashes feature class

In [ ]:
# Apply the metadata object to the crashes feature class
mdCrashes = md.Metadata(crashes)
if not mdCrashes.isReadOnly:
    mdCrashes.copy(mdoCrashes)
    mdCrashes.save()
    print(f"Metadata updated for {crashesAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Parties Metadata</h2>

Create a new metadata object for the parties feature class

In [ ]:
# Define key metadata attributes for the Parties feature class
mdoParties = md.Metadata()
mdoParties.title = "OCSWITRS Parties Points"
mdoParties.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Parties, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoParties.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Parties Data for Orange County, California ({mdYears})"
mdoParties.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on parties involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoParties.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoParties.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoParties.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/1e07bb1002f9457fa6fd3540fdb08e29/data"

Assign the parties metadata object to the parties feature class

In [ ]:
# Apply the metadata object to the parties feature class
mdParties = md.Metadata(parties)
if not mdParties.isReadOnly:
    mdParties.copy(mdoParties)
    mdParties.save()
    print(f"Metadata updated for {partiesAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.4. Victims Metadata</h2>

Create a new metadata object for the victims feature class

In [ ]:
# Define key metadata attributes for the Victims feature class
mdoVictims = md.Metadata()
mdoVictims.title = "OCSWITRS Victims Points"
mdoVictims.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Victims, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoVictims.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Victims Data for Orange County, California ({mdYears})"
mdoVictims.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on victims/persons involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoVictims.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoVictims.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoVictims.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"

Assign the victims metadata object to the victims feature class

In [ ]:
# Apply the metadata object to the victims feature class
mdVictims = md.Metadata(victims)
if not mdVictims.isReadOnly:
    mdVictims.copy(mdoVictims)
    mdVictims.save()
    print(f"Metadata updated for {victimsAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.5. Roads Metadata</h2>

Create a new metadata object for the roads feature class

In [ ]:
# Define key metadata attributes for the Roads feature class
mdoRoads = md.Metadata()
mdoRoads.title = "OCSWITRS Roads Network"
mdoRoads.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoRoads.summary = "All roads for Orange County, California (Primary roads and highways, secondary roads, and local roads)"
mdoRoads.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County Roads Network is a comprehensive representation of all roads in the area, including primary roads and highways, secondary roads, and local roads. The data are sourced from the Orange County Department of Public Works and are updated regularly to reflect the most current road network configuration.</span></p></div></div></div>"""
mdoRoads.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoRoads.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoRoads.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

Assign the roads metadata object to the roads feature class

In [ ]:
# Apply the metadata object to the roads feature class
mdRoads = md.Metadata(roads)
if not mdRoads.isReadOnly:
    mdRoads.copy(mdoRoads)
    mdRoads.save()
    print(f"Metadata updated for {roadsAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.6. Census Blocks Metadata</h2>

Create a new metadata object for the census blocks feature class

In [ ]:
# Define key metadata attributes for the US Census 2020 Blocks feature class
mdoCensusBlocks = md.Metadata()
mdoCensusBlocks.title = "OCSWITRS US Census 2020 Blocks"
mdoCensusBlocks.tags = "Orange County, California, US Census 2020, Blocks, Census, Demographics, Population"
mdoCensusBlocks.summary = "US Census 2020 Blocks for Orange County, California"
mdoCensusBlocks.description = """<div style="text-align:Left;"><div><div><p><span>The US Census 2020 Blocks feature class provides a comprehensive representation of the 2020 Census Blocks for Orange County, California. The data are sourced from the US Census Bureau and are updated regularly to reflect the most current demographic and population data.</span></p></div></div></div>"""
mdoCensusBlocks.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCensusBlocks.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCensusBlocks.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/e2c4cd39783a4d1bb0925ead15a23cdc/data"

Assign the census blocks metadata object to the census blocks feature class

In [ ]:
# Apply the metadata object to the US Census 2020 Blocks feature class
mdCensusBlocks = md.Metadata(censusBlocks)
if not mdCensusBlocks.isReadOnly:
    mdCensusBlocks.copy(mdoCensusBlocks)
    mdCensusBlocks.save()
    print(f"Metadata updated for {censusBlocksAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.7. Cities Metadata</h2>

Create a new metadata object for the cities feature class

In [ ]:
# Define key metadata attributes for the Cities feature class
mdoCities = md.Metadata()
mdoCities.title = "OCSWITRS Cities Boundaries"
mdoCities.tags = "Orange County, California, Cities, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoCities.summary = "Orange County City and Unincorporated Areas Land Boundaries, enriched with geodemographic characteristics"
mdoCities.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County City and Unincorporated Areas Land Boundaries are enriched with a comprehensive set of geodemographic characteristics from OC ACS 2021 data. These characteristics span across demographic, housing, economic, and social aspects, providing a holistic view of the area. </span></p><p><span>The geodemographic data originate from the US Census American Community Survey (ACS) 2021, a 5-year estimate of the key Characteristics of Cities' geographic level in Orange County, California. The data contains:</span></p><ul><li><p><span>Total population and housing counts for each area;</span></p></li><li><p><span>Population and housing density measurements (per square mile);</span></p></li><li><p><span>Race counts for Asian, Black or African American, Hispanic and White groups;</span></p></li><li><p><span>Aggregate values for the number of vehicles commuting and travel time to work;</span></p></li></ul></div></div></div>"""
mdoCities.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCities.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCities.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/ffe4a73307a245eda7dc7eaffe1db6d2/data"

Assign the cities metadata object to the cities feature class

In [ ]:
# Apply the metadata object to the Cities feature class
mdCities = md.Metadata(cities)
if not mdCities.isReadOnly:
    mdCities.copy(mdoCities)
    mdCities.save()
    print(f"Metadata updated for {citiesAlias} feature class.")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.8. Boundaries Metadata</h2>

Create a new metadata object for the boundaries feature class

In [ ]:
# Define key metadata attributes for the Boundaries feature class
mdoBoundaries = md.Metadata()
mdoBoundaries.title = "OC Land Boundaries"
mdoBoundaries.tags = "Orange County, California, Boundaries, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoBoundaries.summary = "Land boundaries for Orange County, cities, and unincorporated areas"
mdoBoundaries.description = """<div style="text-align:Left;"><div><div><p><span>Land boundaries for Orange County, cities, and unincorporated areas (based on the five supervisorial districts). Contains additional geodemographic data on population and housing from the US Census 2021 American Community Survey (ACS).</span></p></div></div></div>"""
mdoBoundaries.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoBoundaries.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoBoundaries.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/4041c4b1f4234218a4ce654e5d22f176/data"

Assign the boundaries metadata object to the boundaries feature class

In [ ]:
# Apply the metadata object to the Boundaries feature class
mdBoundaries = md.Metadata(boundaries)
if not mdBoundaries.isReadOnly:
    mdBoundaries.copy(mdoBoundaries)
    mdBoundaries.save()
    print(f"Metadata updated for {boundariesAlias} feature class.")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [ ]:
# Save the project
aprx.save()

<div style = "background-color:indigo"><center>
<h1 style="font-weight:bold; color:goldenrod; border-top: 2px solid goldenrod; border-bottom: 2px solid goldenrod; padding-top: 5px; padding-bottom: 10px">End of Script</h1>
</center></div>